In [3]:
import pandas as pd
import numpy as np

## user data

In [2]:
#def user_graph():
users  = pd.read_csv('./clean/accounts.csv', on_bad_lines='skip', low_memory=False, lineterminator='\n') #We are encoutering some bad lines creating some error so we skip over them
cat=["Id","Gender", "Social network", "Social network"]
num=['Country id','City id','Gender','Social network','Followers',"Engagement","Posts","Estimated reach","Avg. views per video", "Avg. comments per post", "Avg. likes per post", "Avg. posts per week"]
n = 2000
clean_df = users[cat+num][:n]

clean_df['label'] = clean_df['Id'].apply(lambda x: f'Account:{x}')
clean_df = clean_df.rename(columns={'City id': 'City','Country id':'Country'})
for i in clean_df.columns:
    clean_df = clean_df.rename(columns={i: i.replace(" ","_")})
#clean_df = clean_df.set_index("Id")

In [3]:
#City-ID mapping
city_ids = pd.read_csv('../SocialTalk 2023/cities.csv', low_memory=False) 

clean_df['City'] = clean_df['City'].map(city_ids.set_index('Id')['Name'].to_dict())
clean_df['City'] = clean_df['City'].str.replace(" ","_") #Replace white space by "_"

In [4]:
#Country-ID mapping
country_ids = pd.read_csv('../SocialTalk 2023/countries.csv', low_memory=False)

clean_df['Country'] = clean_df['Country'].map(country_ids.set_index('Id')['Name'].to_dict())
clean_df['Country'] = clean_df['Country'].str.replace(" ","") #Replace white space by "_"

## gender dist

In [5]:
def BIN(x,i):
    if x == "Unknown":
        return "Unknown"
    if(x < audience_genders.describe().loc['25%'][i]):
        return 'Low'
    elif(x >audience_genders.describe().loc['25%'][i] and x<audience_genders.describe().loc['75%'][i]):
        return 'Medium'
    elif(x>audience_genders.describe().loc['75%'][i]):
        return 'High'

In [6]:
#changing the column names
audience_genders = pd.read_csv('./clean/accounts-audience-genders.csv')[:n]
for i in audience_genders.columns[1:]:
    audience_genders = audience_genders.rename(columns={i: i.replace(" ","_")})


In [7]:
#binning the data
for c in audience_genders.columns[1:]:
    audience_genders[c] = audience_genders[c].apply(lambda x: BIN(x,c))


In [8]:
#merging audience gender distribution and accounts data
clean_df = clean_df.merge(audience_genders,how='left' ,left_on='Id',right_on = 'Account Id')

In [9]:
clean_df.isna().sum()

Id                           0
Gender                       0
Social_network               0
Social_network               0
Country                     70
City                      1157
Gender                       0
Social_network               0
Followers                    0
Engagement                   0
Posts                        0
Estimated_reach              0
Avg._views_per_video         0
Avg._comments_per_post       0
Avg._likes_per_post          0
Avg._posts_per_week          0
label                        0
Account Id                 680
MALE_%                     680
FEMALE_%                   680
dtype: int64

In [10]:
#dropping "Account Id" from the table and filling na wuth "Unknown"
clean_df = clean_df.drop(['Account Id'], axis=1)
clean_df = clean_df.fillna("Unknown")

In [11]:
clean_df.isna().sum()

Id                        0
Gender                    0
Social_network            0
Social_network            0
Country                   0
City                      0
Gender                    0
Social_network            0
Followers                 0
Engagement                0
Posts                     0
Estimated_reach           0
Avg._views_per_video      0
Avg._comments_per_post    0
Avg._likes_per_post       0
Avg._posts_per_week       0
label                     0
MALE_%                    0
FEMALE_%                  0
dtype: int64

## Age dist

In [5]:
audience_ages

,Account Id,13-17 %,18-24 %,25-34 %,35-44 %,45-64 %,65- %
0,151,0.000000,0.080247,0.370829,0.436014,0.112910,0.000000
1,152,0.065500,0.411617,0.423468,0.084491,0.014754,0.000170
2,153,0.059758,0.372736,0.422436,0.116510,0.028560,0.000000
3,154,0.061143,0.362597,0.420467,0.126529,0.028899,0.000365
4,155,0.181900,0.533887,0.236461,0.035696,0.011503,0.000553
...,...,...,...,...,...,...,...
2657891,2936780,0.044621,0.356476,0.447003,0.126717,0.025002,0.000182
2657892,2936781,0.058716,0.385223,0.437200,0.082120,0.036741,0.000000
2657893,2936782,0.012735,0.179177,0.414442,0.292934,0.100712,0.000000
2657894,2936783,0.019021,0.257128,0.527971,0.167132,0.028294,0.000453


In [4]:
audience_ages = pd.read_csv('./clean/accounts-audience-ages.csv')
audience_ages = audience_ages[:n]
for i in audience_ages.columns:
    if(i != 'Account Id'):
        audience_ages.rename(columns = {i:'Audience-Age-' + i.replace(' ','_')}, inplace = True)

NameError: name 'n' is not defined

In [13]:
#mapping accounts and age distribution
clean_df = clean_df.merge(audience_ages,how='left' ,left_on='Id',right_on = 'Account Id')
clean_df.isna().sum()

Id                           0
Gender                       0
Social_network               0
Social_network               0
Country                      0
City                         0
Gender                       0
Social_network               0
Followers                    0
Engagement                   0
Posts                        0
Estimated_reach              0
Avg._views_per_video         0
Avg._comments_per_post       0
Avg._likes_per_post          0
Avg._posts_per_week          0
label                        0
MALE_%                       0
FEMALE_%                     0
Audience-Age-Unnamed:_0    713
Account Id                 713
Audience-Age-13-17_%       713
Audience-Age-18-24_%       713
Audience-Age-25-34_%       713
Audience-Age-35-44_%       713
Audience-Age-45-64_%       713
Audience-Age-65-_%         713
dtype: int64

In [14]:
clean_df = clean_df.drop(['Account Id',"Audience-Age-Unnamed:_0"], axis=1)
clean_df = clean_df.fillna(-1)

In [15]:
for i in range(len(clean_df)):
    for c in audience_ages.columns[2:]:
        if clean_df.loc[i,c] !=-1:
            clean_df.loc[i,c] *= clean_df.loc[i,"Followers"]

In [16]:
clean_df

,Id,Gender,Social_network,Social_network,Country,City,Gender,Social_network,Followers,Engagement,...,Avg._posts_per_week,label,MALE_%,FEMALE_%,Audience-Age-13-17_%,Audience-Age-18-24_%,Audience-Age-25-34_%,Audience-Age-35-44_%,Audience-Age-45-64_%,Audience-Age-65-_%
0,151,Male,Instagram,Instagram,Spain,Lisbon,Male,Instagram,154,0,...,1,Account:151,Medium,Medium,0.000000e+00,1.235804e+01,5.710767e+01,6.714616e+01,1.738814e+01,0.000000
1,152,Male,Instagram,Instagram,UnitedStates,Los_Angeles,Male,Instagram,22625181,39322588,...,9,Account:152,Medium,Medium,1.481949e+06,9.312909e+06,9.581040e+06,1.911624e+06,3.338119e+05,3846.280770
2,153,Male,Instagram,Instagram,SaudiArabia,Manchester,Male,Instagram,550332150,1032630249,...,4,Account:153,High,Low,3.288675e+07,2.051286e+08,2.324801e+08,6.411920e+07,1.571749e+07,0.000000
3,154,Male,Instagram,Instagram,Brazil,São_Paulo,Male,Instagram,71552085,59144108,...,4,Account:154,High,Low,4.374909e+06,2.594457e+07,3.008529e+07,9.053414e+06,2.067784e+06,26116.511025
4,155,Female,Instagram,Instagram,UnitedStates,Unknown,Female,Instagram,39861444,17922824,...,2,Account:155,Medium,Medium,7.250797e+06,2.128151e+07,9.425677e+06,1.422894e+06,4.585262e+05,22043.378532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2210,Female,Instagram,Instagram,Spain,Valencia,Female,Instagram,28558,7926,...,4,Account:2210,Medium,Medium,4.136998e+02,3.162543e+03,8.879231e+03,8.294947e+03,7.667715e+03,139.864391
1996,2211,Female,Instagram,Instagram,Spain,Madrid,Female,Instagram,26990,9248,...,2,Account:2211,High,Low,8.104089e+02,5.620698e+03,1.132625e+04,5.673864e+03,3.506708e+03,52.072749
1997,2212,Female,Instagram,Instagram,Spain,Jaén,Female,Instagram,45908,98057,...,50,Account:2212,Unknown,Unknown,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000
1998,2213,Female,Instagram,Instagram,Spain,Barcelona,Female,Instagram,24073,6766,...,1,Account:2213,Medium,Medium,1.616102e+03,9.870624e+03,9.722985e+03,2.173605e+03,6.797614e+02,9.923266


## graph

In [17]:
numerics = [ 'int64', 'float64']
num_df=clean_df.select_dtypes(include=numerics)

In [18]:
#numerical edges
for i in num_df.columns[1:]:
    clean_df[i] = pd.qcut(clean_df[i],q=3,duplicates='drop')

In [19]:
clean_df

,Id,Gender,Social_network,Social_network,Country,City,Gender,Social_network,Followers,Engagement,...,Avg._posts_per_week,label,MALE_%,FEMALE_%,Audience-Age-13-17_%,Audience-Age-18-24_%,Audience-Age-25-34_%,Audience-Age-35-44_%,Audience-Age-45-64_%,Audience-Age-65-_%
0,151,Male,Instagram,Instagram,Spain,Lisbon,Male,Instagram,"(-0.001, 7188.667]","(-0.001, 4611.667]",...,"(-0.001, 1.0]",Account:151,Medium,Medium,"(-1.0, 695.517]","(-1.001, 5044.372]","(-1.001, 8081.292]","(-1.001, 2979.189]","(-1.001, 1179.71]","(-1.001, 9.408]"
1,152,Male,Instagram,Instagram,UnitedStates,Los_Angeles,Male,Instagram,"(32820.667, 550332150.0]","(19693.333, 1032630249.0]",...,"(3.0, 224.0]",Account:152,Medium,Medium,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(9.408, 72428.53]"
2,153,Male,Instagram,Instagram,SaudiArabia,Manchester,Male,Instagram,"(32820.667, 550332150.0]","(19693.333, 1032630249.0]",...,"(3.0, 224.0]",Account:153,High,Low,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(-1.001, 9.408]"
3,154,Male,Instagram,Instagram,Brazil,São_Paulo,Male,Instagram,"(32820.667, 550332150.0]","(19693.333, 1032630249.0]",...,"(3.0, 224.0]",Account:154,High,Low,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(9.408, 72428.53]"
4,155,Female,Instagram,Instagram,UnitedStates,Unknown,Female,Instagram,"(32820.667, 550332150.0]","(19693.333, 1032630249.0]",...,"(1.0, 3.0]",Account:155,Medium,Medium,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(9.408, 72428.53]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2210,Female,Instagram,Instagram,Spain,Valencia,Female,Instagram,"(7188.667, 32820.667]","(4611.667, 19693.333]",...,"(3.0, 224.0]",Account:2210,Medium,Medium,"(-1.0, 695.517]","(-1.001, 5044.372]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(9.408, 72428.53]"
1996,2211,Female,Instagram,Instagram,Spain,Madrid,Female,Instagram,"(7188.667, 32820.667]","(4611.667, 19693.333]",...,"(1.0, 3.0]",Account:2211,High,Low,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(2979.189, 64119198.797]","(1179.71, 15717486.204]","(9.408, 72428.53]"
1997,2212,Female,Instagram,Instagram,Spain,Jaén,Female,Instagram,"(32820.667, 550332150.0]","(19693.333, 1032630249.0]",...,"(3.0, 224.0]",Account:2212,Unknown,Unknown,"(-157892.97199999998, -1.0]","(-1.001, 5044.372]","(-1.001, 8081.292]","(-1.001, 2979.189]","(-1.001, 1179.71]","(-1.001, 9.408]"
1998,2213,Female,Instagram,Instagram,Spain,Barcelona,Female,Instagram,"(7188.667, 32820.667]","(4611.667, 19693.333]",...,"(-0.001, 1.0]",Account:2213,Medium,Medium,"(695.517, 32886748.62]","(5044.372, 205128604.262]","(8081.292, 232480112.117]","(-1.001, 2979.189]","(-1.001, 1179.71]","(9.408, 72428.53]"


In [20]:
## creating edges from users data
edges = []
for column in clean_df.columns:
    series_tuples = [
        (account, f'{column}:{value}') for account, value in clean_df[column].items()
        ]
    edges.extend(series_tuples)

In [21]:
## removing all the edges that have value as low (numerical columns)
e2 = [i for i in edges if not i[1].endswith('low')]
len(e2)

40013

In [22]:
temp = pd.DataFrame(edges, columns=['source', 'target'])
temp.to_csv(f'edge_list_{len(edges)}.csv',index=False)

In [28]:
from gensim.models import KeyedVectors
from nodevectors import Node2Vec, GGVec
import networkx as nx

G = nx.from_pandas_edgelist(temp)
g2v = Node2Vec(verbose=True)
n2v_embeddings = g2v.fit(G)
g2v.save_vectors("wheel_model.bin")

Making walks... Done, T=3.05
Mapping Walk Names... Done, T=1.40
Training W2V... WARNING: gensim word2vec version is unoptimizedTry version 3.6 if on windows, versions 3.7 and 3.8 have had issues
Done, T=66.03


In [38]:
words = list(g2v.model.wv.vocab)
g2v_embeds = pd.DataFrame([g2v.model.wv[i] for i in words ], index=words)

In [40]:
index = []
for i in g2v_embeds.index:
    if i.split(":")[0] == 'label':
        index.append(i)

In [43]:
g2v_embeds[index]

KeyError: "None of [Index(['label:Account:2143', 'label:Account:475', 'label:Account:166',\n       'label:Account:787', 'label:Account:1047', 'label:Account:469',\n       'label:Account:795', 'label:Account:683', 'label:Account:1152',\n       'label:Account:159',\n       ...\n       'label:Account:2092', 'label:Account:2207', 'label:Account:2109',\n       'label:Account:2120', 'label:Account:2127', 'label:Account:2129',\n       'label:Account:2144', 'label:Account:2161', 'label:Account:2165',\n       'label:Account:2180'],\n      dtype='object', length=2000)] are in the [columns]"